In [1]:
from datetime import datetime
from urllib.error import HTTPError

from langchain_core.messages.tool import tool_call
from langchain_core.prompts import ChatPromptTemplate
from openai import api_key

print("Hello, world!")

Hello, world!


In [2]:
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 1.7 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 2.171.0
    Uninstalling google-api-python-client-2.171.0:
      Successfully uninstalled google-api-python-client-2.171.0


In [3]:
!pip install google-ai-generativelanguage==0.6.15

  Using cached google_ai_generativelanguage-0.6.15-py3-none-any.whl.metadata (5.7 kB)
Using cached google_ai_generativelanguage-0.6.15-py3-none-any.whl (1.3 MB)
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.18
    Uninstalling google-ai-generativelanguage-0.6.18:
      Successfully uninstalled google-ai-generativelanguage-0.6.18
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-google-genai 2.1.5 requires google-ai-generativelanguage<0.7.0,>=0.6.18, but you have google-ai-generativelanguage 0.6.15 which is incompatible.


In [4]:
# # Install all required packages with specific versions in Colab
# !pip install google-api-python-client==2.171.0 \
# google-auth-httplib2==0.2.0 \
#     google-auth-oauthlib==1.2.2 \
#     google-ai-generativelanguage==0.6.15 \
#     google-generativeai==0.8.5

In [1]:
import os
import datetime
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.errors import HttpError
from langchain_core.tools import tool

SCOPES = ["https://www.googleapis.com/auth/calendar.readonly","https://www.googleapis.com/auth/calendar"]

def parse_date_string(date_str: str) -> tuple:
    """Parse the date string with before/after parameters."""
    days_before = 2
    days_after = 2

    parts = date_str.split('/')
    date_part = parts[0]

    for part in parts[1:]:
        if part.startswith("before="):
            try:
                days_before = int(part.split('=')[1])
            except (IndexError, ValueError):
                print(f"Warning: Invalid before value in {part}, using default {days_before}")
        elif part.startswith("after="):
            try:
                days_after = int(part.split('=')[1])
            except (IndexError, ValueError):
                print(f"Warning: Invalid after value in {part}, using default {days_after}")

    return date_part, days_before, days_after

@tool
def read_calendar(date_str: str) -> list:
    """Read calendar events within a date range with enhanced debugging."""
    try:
        date, days_before, days_after = parse_date_string(date_str)
    except Exception as e:
        return [{"error": f"Invalid date format: {str(e)}. Use YYYY-MM-DD/before=X/after=Y"}]

    # Authentication
    creds = None
    token_file = "token.json"
    creds_file = "credentials.json"

    if not os.path.exists(creds_file):
        return [{"error": f"Credentials file not found at {creds_file}"}]

    if os.path.exists(token_file):
        creds = Credentials.from_authorized_user_file(token_file, SCOPES)

    if not creds or not creds.valid:
        try:
            if creds and creds.expired and creds.refresh_token:
                creds.refresh(Request())
            else:
                flow = InstalledAppFlow.from_client_secrets_file(creds_file, SCOPES)
                creds = flow.run_local_server(port=8088)
            with open(token_file, "w") as token:
                token.write(creds.to_json())
        except Exception as auth_error:
            return [{"error": f"Authentication failed: {str(auth_error)}"}]

    try:
        service = build("calendar", "v3", credentials=creds)
        all_events = []

        # Set timezone to IST (UTC+5:30)
        tz = datetime.timezone(datetime.timedelta(hours=5, minutes=30))
        try:
            central_date = datetime.datetime.strptime(date, "%Y-%m-%d").replace(tzinfo=tz)
        except ValueError:
            return [{"error": "Invalid date format. Please use YYYY-MM-DD"}]

        # Calculate date range (inclusive of whole days)
        start_date = (central_date - datetime.timedelta(days=days_before)).replace(
            hour=0, minute=0, second=0, microsecond=0)
        end_date = (central_date + datetime.timedelta(days=days_after + 1)).replace(
            hour=0, minute=0, second=0, microsecond=0)

        print(f"\nSearching events from {start_date.date()} to {end_date.date()-datetime.timedelta(days=1)}")

        # Get list of all calendars with more details
        try:
            calendar_list = service.calendarList().list(
                minAccessRole="reader",
                showHidden=True
            ).execute()
            calendars = calendar_list.get('items', [])
            print(f"\nFound {len(calendars)} calendars:")
            for cal in calendars:
                print(f" - {cal.get('summary', 'Unnamed')} (ID: {cal['id']})")
        except HttpError as e:
            print(f"\nError getting calendars: {e}")
            calendars = []

        # Always check primary calendar first
        calendars.insert(0, {'id': 'primary', 'summary': 'Primary Calendar'})

        for calendar in calendars:
            calendar_id = calendar['id']
            calendar_name = calendar.get('summary', 'Unnamed Calendar')
            print(f"\nChecking calendar: {calendar_name} ({calendar_id})")

            try:
                events_result = service.events().list(
                    calendarId=calendar_id,
                    timeMin=start_date.isoformat(),
                    timeMax=end_date.isoformat(),
                    singleEvents=True,
                    orderBy="startTime",
                    showDeleted=False
                ).execute()

                events = events_result.get('items', [])
                print(f"Found {len(events)} events in {calendar_name}")

                for event in events:
                    # Skip cancelled events
                    if event.get('status') == 'cancelled':
                        continue

                    # Get event time (handles both all-day and timed events)
                    start = event.get('start', {})
                    end = event.get('end', {})

                    event_date = start.get('dateTime', start.get('date'))
                    if not event_date:
                        continue

                    formatted_event = {
                        'summary': event.get('summary', 'No title'),
                        'start': start,
                        'end': end,
                        'calendar': calendar_name,
                        'date': event_date[:10],
                        'description': event.get('description', '')[:100] + '...' if event.get('description') else '',
                        'location': event.get('location', ''),
                        'status': event.get('status', 'confirmed')
                    }
                    all_events.append(formatted_event)

                    # Debug print for the event we're looking for
                    if 'thug life' in formatted_event['summary'].lower():
                        print("\nFOUND THUG LIFE EVENT:")
                        print(formatted_event)

            except HttpError as e:
                print(f"\nError accessing calendar {calendar_name}: {e}")
                continue

        if not all_events:
            return [{
                'summary': f'No events found between {start_date.date()} and {end_date.date()-datetime.timedelta(days=1)}',
                'date_range': {
                    'start': start_date.date().isoformat(),
                    'end': (end_date.date()-datetime.timedelta(days=1)).isoformat()
                },
                'calendars_checked': len(calendars),
                'note': 'Try expanding the date range or check if the event exists in another calendar'
            }]

        # Sort events by date
        all_events.sort(key=lambda x: x['date'])

        # Debug output
        print("\nAll found events:")
        for event in all_events:
            print(f"{event['date']} - {event['summary']} ({event['calendar']})")

        return all_events

    except HttpError as e:
        return [{"error": f"Calendar API error: {str(e)}"}]
    except Exception as e:
        return [{"error": f"Unexpected error: {str(e)}"}]

In [2]:
# Example usage
if __name__ == "__main__":
    events = read_calendar("2025-10-11/before=10/after=10")

    print("\nResults:")
    for event in events:
        if 'error' in event:
            print(f"Error: {event['error']}")
        elif 'start' in event:
            start = event["start"].get("dateTime", event["start"].get("date", "No start time"))
            print(f"{start}: {event['summary']} (Calendar: {event.get('calendar', 'Unknown')})")
        else:
            print(f"No events found in the specified date range")

/tmp/ipykernel_8410/4179805440.py:3: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  events = read_calendar("2025-10-11/before=10/after=10")


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=1054231560631-pn31nni4vam9kj6iid46bqr5pft15amc.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8088%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar.readonly+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar&state=7jHw6mB22HlrAkR2jkfyIKE6zqtf2U&access_type=offline
Opening in existing browser session.

Searching events from 2025-10-01 to 2025-10-21

Found 2 calendars:
 - sharmasuhas450@gmail.com (ID: sharmasuhas450@gmail.com)
 - Formula 1 (ID: 021e4d479b307dc5dd102935344b2d369847e90263f69b783556cb3a8bf489cd@group.calendar.google.com)

Checking calendar: Primary Calendar (primary)
Found 3 events in Primary Calendar

Checking calendar: sharmasuhas450@gmail.com (sharmasuhas450@gmail.com)
Found 3 events in sharmasuhas450@gmail.com

Checking calendar: Formula 1 (021e4d479b307dc5dd102935344b2d369847e90263f69b783556cb3a8bf489cd@group.calendar.

In [14]:
@tool
def create_event(event_data: Dict[str, Any]) -> Dict[str, Any]:
    """Create a new calendar event with comprehensive error handling and timezone support.

    Features:
    - Automatic timezone handling (defaults to UTC+5:30 for India)
    - Supports both timed events and all-day events
    - Detailed error reporting
    - Automatic retry on authentication failure
    - Verbose logging for debugging

    Args:
        event_data: Dictionary containing event details with these fields:
            Required:
            - summary: Title of the event (string)
            - start: Start time in format:
                * 'YYYY-MM-DD' for all-day events
                * 'YYYY-MM-DDTHH:MM:SS' for timed events (will auto-add timezone)
                * {'dateTime': '...', 'timeZone': '...'} for explicit timezone
            - end: End time in same format as start

            Optional:
            - description: Event description (string)
            - location: Physical location (string)
            - attendees: List of email dictionaries:
                [{'email': 'user@example.com'}, ...]
            - reminders: Dictionary with either:
                * {'useDefault': True} or
                * {'overrides': [{'method': 'popup', 'minutes': 30}, ...]}
            - colorId: Event color ID (1-11)
            - visibility: 'public', 'private', or 'confidential'
            - recurrence: RRULE string array (e.g., ['RRULE:FREQ=DAILY;COUNT=2'])
            - extendedProperties: Dictionary for custom metadata

    Returns:
        Dictionary with:
        - On success:
            {
                'status': 'success',
                'event_link': 'https://calendar.google.com/...',
                'event_id': 'google_event_id',
                'summary': 'Event title',
                'start': 'formatted_start',
                'end': 'formatted_end',
                'calendar': 'primary'
            }
        - On error:
            {
                'status': 'error',
                'error': 'Error description',
                'details': {additional error info if available}
            }
    """
    # Authentication setup (similar to read_calendar)
    creds = None
    token_file = "token.json"
    creds_file = "credentials.json"

    if not os.path.exists(creds_file):
        return {
            'status': 'error',
            'error': 'Missing credentials',
            'details': f'Credentials file not found at {creds_file}'
        }

    # Load or refresh credentials
    try:
        if os.path.exists(token_file):
            creds = Credentials.from_authorized_user_file(token_file, EDITSCOPES)

        if not creds or not creds.valid:
            if creds and creds.expired and creds.refresh_token:
                creds.refresh(Request())
            else:
                flow = InstalledAppFlow.from_client_secrets_file(creds_file, EDITSCOPES)
                creds = flow.run_local_server(port=8088)

            with open(token_file, "w") as token:
                token.write(creds.to_json())
    except Exception as auth_error:
        return {
            'status': 'error',
            'error': 'Authentication failed',
            'details': str(auth_error)
        }

    # Prepare the event with proper time formatting
    formatted_event = event_data.copy()
    timezone = datetime.timezone(datetime.timedelta(hours=5, minutes=30))  # IST

    for time_field in ['start', 'end']:
        if time_field in formatted_event:
            # Case 1: Already properly formatted
            if isinstance(formatted_event[time_field], dict):
                continue

            # Case 2: Date string (all-day event)
            elif 'T' not in formatted_event[time_field]:
                formatted_event[time_field] = {
                    'date': formatted_event[time_field]
                }

            # Case 3: DateTime string without timezone
            else:
                try:
                    dt = datetime.datetime.strptime(
                        formatted_event[time_field],
                        '%Y-%m-%dT%H:%M:%S'
                    ).replace(tzinfo=timezone)
                    formatted_event[time_field] = {
                        'dateTime': dt.isoformat(),
                        'timeZone': 'Asia/Kolkata'
                    }
                except ValueError:
                    return {
                        'status': 'error',
                        'error': 'Invalid time format',
                        'details': f'{time_field} must be YYYY-MM-DD or YYYY-MM-DDTHH:MM:SS'
                    }

    # Add default reminders if none specified
    if 'reminders' not in formatted_event:
        formatted_event['reminders'] = {
            'useDefault': True
        }

    # Create the event
    try:
        service = build("calendar", "v3", credentials=creds)

        print(f"\nCreating event with details:")
        for key, value in formatted_event.items():
            print(f" - {key}: {str(value)[:100]}{'...' if len(str(value)) > 100 else ''}")

        created_event = service.events().insert(
            calendarId='primary',
            body=formatted_event,
            sendUpdates='none'  # Change to 'all' to notify attendees
        ).execute()

        print(f"\nSuccessfully created event: {created_event.get('htmlLink')}")

        return {
            'status': 'success',
            'event_link': created_event.get('htmlLink'),
            'event_id': created_event['id'],
            'summary': created_event.get('summary'),
            'start': created_event.get('start'),
            'end': created_event.get('end'),
            'calendar': 'primary'
        }

    except HttpError as e:
        error_details = {
            'status_code': e.resp.status,
            'reason': e._get_reason(),
            'error_details': e.error_details
        }
        return {
            'status': 'error',
            'error': 'Calendar API error',
            'details': error_details
        }
    except Exception as e:
        return {
            'status': 'error',
            'error': 'Unexpected error',
            'details': str(e)
        }

In [18]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import  AgentExecutor, create_tool_calling_agent
from langchain.prompts import ChatPromptTemplate,MessagesPlaceholder
tools=[create_event, read_calendar]
llm=ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.2, max_output_tokens=1000)
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that can read and create calendar events."),
    ("human", "{input}"),
    MessagesPlaceholder("agent_scratchpad"),
])
agent=create_tool_calling_agent(llm, tools=tools,prompt=prompt)

Agent=AgentExecutor(agent=agent, tools=tools, verbose=True)

Agent.invoke({"input": """



Conversation opened. 1 read message.

Skip to content
Using Gmail with screen readers
events
2 of many
Registration confirmed for Kotlin & Koffee 3rd Edition
Inbox
KUG Chennai Events <kugchennai@calendar.luma-mail.com>
Attachments
Sat 24 May, 10:11
to me

This event isn't in your calendar yet You haven't interacted with kugchennai@calendar.luma-mail.com before. Do you want to automatically add this and future invitations from them to your calendar?
Add to calendarReport spam
May31Sat
Kotlin & Koffee 3rd Edition
View on Google Calendar
When	Sat 31 May 2025 9:30am – 12:30pm (IST)
Where	Discord
Who	KUG Chennai Events*

KUG Chennai Events
You have registered for
Kotlin & Koffee 3rd Edition
MAY
31
Saturday 31 May
9:30 am - 12:30 pm IST

Discord ↗
Event Page
My Ticket
 
Share with friends

















Host your event with Luma ↗
 
 2 attachments
  • Scanned by Gmail
Emily Logo
AI Reply


i have given you the contents of the email and you are asked to clean the email extract the required contents and create an event on that partiular day of event check fo rnay other events on that day if there are any other evets odnt to anything if there are no events create an event with the details provided in the email and send me the link to the event created
"""})



> Entering new AgentExecutor chain...

Invoking: `create_event` with `{'event_data': {'end': {'timeZone': 'Asia/Kolkata', 'dateTime': '2025-05-31T12:30:00'}, 'start': {'timeZone': 'Asia/Kolkata', 'dateTime': '2025-05-31T09:30:00'}, 'location': 'Discord', 'summary': 'Kotlin & Koffee 3rd Edition'}}`



Creating event with details:
 - end: {'timeZone': 'Asia/Kolkata', 'dateTime': '2025-05-31T12:30:00'}
 - start: {'timeZone': 'Asia/Kolkata', 'dateTime': '2025-05-31T09:30:00'}
 - location: Discord
 - summary: Kotlin & Koffee 3rd Edition
 - reminders: {'useDefault': True}

Successfully created event: https://www.google.com/calendar/event?eid=bW1lYTltNzI2bDYxc3ZiZTZmZWw1ZGs2Yjggc2hhcm1hc3VoYXM0NTBAbQ
{'status': 'success', 'event_link': 'https://www.google.com/calendar/event?eid=bW1lYTltNzI2bDYxc3ZiZTZmZWw1ZGs2Yjggc2hhcm1hc3VoYXM0NTBAbQ', 'event_id': 'mmea9m726l61svbe6fel5dk6b8', 'summary': 'Kotlin & Koffee 3rd Edition', 'start': {'dateTime': '2025-05-31T09:30:00+05:30', 'timeZone': 'Asia/Ko

{'input': "\n\n\n\nConversation opened. 1 read message.\n\nSkip to content\nUsing Gmail with screen readers\nevents\n2 of many\nRegistration confirmed for Kotlin & Koffee 3rd Edition\nInbox\nKUG Chennai Events <kugchennai@calendar.luma-mail.com>\nAttachments\nSat 24 May, 10:11\nto me\n\nThis event isn't in your calendar yet You haven't interacted with kugchennai@calendar.luma-mail.com before. Do you want to automatically add this and future invitations from them to your calendar?\nAdd to calendarReport spam\nMay31Sat\nKotlin & Koffee 3rd Edition\nView on Google Calendar\nWhen\tSat 31 May 2025 9:30am – 12:30pm (IST)\nWhere\tDiscord\nWho\tKUG Chennai Events*\n\nKUG Chennai Events\nYou have registered for\nKotlin & Koffee 3rd Edition\nMAY\n31\nSaturday 31 May\n9:30 am - 12:30 pm IST\n\nDiscord ↗\nEvent Page\nMy Ticket\n\u200a\nShare with friends\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nHost your event with Luma ↗\n\u200a\n 2 attachments\n  • Scanned by Gmail\nEmily Logo\nAI Reply\n\n\ni have gi